In [3]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "admin.json"

In [4]:
PROJECT_ID = "mlops-433612"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

API_ENDPOINT = f"{LOCATION}-aiplatform.googleapis.com"

from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [5]:
from google.cloud.aiplatform_v1beta1 import (
    FeatureOnlineStoreAdminServiceClient, FeatureRegistryServiceClient)

admin_client = FeatureOnlineStoreAdminServiceClient(
    client_options={"api_endpoint": API_ENDPOINT}
)
registry_client = FeatureRegistryServiceClient(
    client_options={"api_endpoint": API_ENDPOINT}
)

In [6]:
from vertexai.resources.preview import FeatureOnlineStore

FEATURE_ONLINE_STORE_ID = "test_online_store"  # @param {type:"string"}

my_fos = FeatureOnlineStore.create_optimized_store(FEATURE_ONLINE_STORE_ID)

2024-09-15 16:22:05.655997: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-15 16:22:05.690533: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-15 16:22:05.723943: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-15 16:22:05.750990: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-15 16:22:05.757612: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-15 16:22:05.781021: I tensorflow/core/platform/cpu_feature_gu

Creating FeatureOnlineStore
Create FeatureOnlineStore backing LRO: projects/994601067814/locations/us-central1/featureOnlineStores/test_online_store/operations/5738785664254607360
FeatureOnlineStore created. Resource name: projects/994601067814/locations/us-central1/featureOnlineStores/test_online_store
To use this FeatureOnlineStore in another session:
feature_online_store = aiplatform.FeatureOnlineStore('projects/994601067814/locations/us-central1/featureOnlineStores/test_online_store')


In [10]:
from vertexai.resources.preview import FeatureViewBigQuerySource

FEATURE_VIEW_ID = "test_fv"  # @param {type:"string"}

CRON_SCHEDULE = "* * * * *"  # @param {type:"string"}

DATASET_ID = "featurestore_demo"
TABLE_ID = "cleaned_table"

BQ_VIEW_ID_FQN = f'{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}'

my_fv = my_fos.create_feature_view(
    FEATURE_VIEW_ID,
    FeatureViewBigQuerySource(
        uri=f"bq://{BQ_VIEW_ID_FQN}",
        entity_id_columns=["entity_id"],
    ),
    # sync_config can be set with CRON_SCHEDULE if you want to setup auto sync
    sync_config=None,
)

Creating FeatureView
Create FeatureView backing LRO: projects/994601067814/locations/us-central1/featureOnlineStores/test_online_store/featureViews/test_fv/operations/4134659771980578816
FeatureView created. Resource name: projects/994601067814/locations/us-central1/featureOnlineStores/test_online_store/featureViews/test_fv
To use this FeatureView in another session:
feature_view = aiplatform.FeatureView('projects/994601067814/locations/us-central1/featureOnlineStores/test_online_store/featureViews/test_fv')


In [11]:
fv_sync = my_fv.sync()

import time

while True:
    feature_view_sync = my_fv.get_sync(fv_sync.resource_name.split("/")[9]).gca_resource
    if feature_view_sync.run_time.end_time.seconds > 0:
        status = "Succeed" if feature_view_sync.final_status.code == 0 else "Failed"
        print(f"Sync {status} for {feature_view_sync.name}.")
        # wait a little more for the job to properly shutdown
        time.sleep(30)
        break
    else:
        print("Sync ongoing, waiting for 30 seconds.")
    time.sleep(30)

Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync Succeed for projects/994601067814/locations/us-central1/featureOnlineStores/test_online_store/featureViews/test_fv/featureViewSyncs/5419906762255892480.


In [17]:
from vertexai.resources.preview import FeatureView
import pandas as pd

# Depends on the FeatureView you created, the FEATURE_VIEW_ID here can be different
data = FeatureView(name=FEATURE_VIEW_ID, feature_online_store_id=FEATURE_ONLINE_STORE_ID).read(key=["10"]).to_dict()

print(data)

Public endpoint for the optimized online store test_online_store is 6340232646500548608.us-central1-994601067814.featurestore.vertexai.goog
{'features': [{'name': 'PassengerId', 'value': {'int64_value': '11'}}, {'name': 'Survived', 'value': {'int64_value': '1'}}, {'name': 'Pclass', 'value': {'int64_value': '3'}}, {'name': 'Name', 'value': {'string_value': 'Sandstrom, Miss. Marguerite Rut'}}, {'name': 'Sex', 'value': {'string_value': 'female'}}, {'name': 'Age', 'value': {'double_value': 4.0}}, {'name': 'SibSp', 'value': {'int64_value': '1'}}, {'name': 'Parch', 'value': {'int64_value': '1'}}, {'name': 'Ticket', 'value': {'string_value': 'PP 9549'}}, {'name': 'Fare', 'value': {'double_value': 16.7}}, {'name': 'Cabin', 'value': {'string_value': 'G6'}}, {'name': 'Embarked', 'value': {'string_value': 'S'}}, {'name': 'feature_timestamp', 'value': {'int64_value': '1726387971197939'}}]}
